In [1]:
import torch.nn as nn
import pandas as pd

class ResNet(nn.Module):
    def __init__(self, num_classes=12):
        super(ResNet, self).__init__()
        self.fc1 = nn.Linear(24, 96)
        self.bn1 = nn.BatchNorm1d(96)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(96, 24)
        self.bn2 = nn.BatchNorm1d(24)
        self.fc3 = nn.Linear(24, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        identity = x
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        out = self.fc3(out)
        out = self.softmax(out)
        return out

In [2]:
df = pd.read_csv('iot23_test_binary.csv')

In [3]:
df['label'].value_counts()

Benign       40051
Malicious    39949
Name: label, dtype: int64

In [4]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]

In [5]:
X.shape

(80000, 24)

In [6]:
Y = pd.get_dummies(df['label']).values

In [7]:
Y.shape

(80000, 2)

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
normalized_x = scaler.transform(X)
scaler.fit(Y)
normalized_y = scaler.transform(Y)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification

# Convert the data to PyTorch tensors

X = torch.tensor(normalized_x, dtype=torch.float32)
Y = torch.tensor(normalized_y, dtype=torch.float32)

# load the ResNet model
model = torch.load('resnet_binary.pt')

# Evaluate the model on the validation set
with torch.no_grad():
    model.eval()
    outputs = model(X)
    _, predicted = torch.max(outputs.data, 1)
    total = Y.size(0)
    _, normal_y_val = torch.max(Y.data,1)
    correct = (predicted == normal_y_val).sum().item()
    accuracy = correct / total
    print('Accuracy on the validation set: {:.2f}%'.format(accuracy * 100))


Accuracy on the validation set: 55.64%
